
# Face Mask Detection: End-to-End Pipeline

This notebook demonstrates the complete pipeline for building a Face Mask Detection system.
We will cover:
1. **Data Loading & Exploration**: Understanding our dataset.
2. **Preprocessing**: Preparing images for the model.
3. **Model Architecture**: Using Transfer Learning with ResNet50.
4. **Training**: Implementing the training loop with metrics.
5. **Evaluation**: Analyzing model performance.
6. **Inference**: Making predictions on new images.

**Train of Thought**:
Throughout this notebook, I will explain the reasoning behind key decisions, such as why we chose specific hyperparameters, loss functions, and model architectures.


In [ ]:

import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import cv2
from pathlib import Path

# Add src to path so we can import our modules
sys.path.append(os.path.abspath('../src'))

from dataset import create_data_loaders, get_transforms
from model import create_model
from train import Trainer
from inference import FaceMaskPredictor
from utils import plot_training_history

%matplotlib inline



## 1. Data Loading & Exploration

First, we need to load our dataset. We assume the data is organized in folders representing classes.
We use `create_data_loaders` from our `src.dataset` module which handles splitting the data into training and validation sets.

**Why split data?**
It's crucial to keep a separate validation set to monitor for overfitting. If the model performs well on training data but poorly on validation data, it's memorizing instead of learning.


In [ ]:

# Configuration
DATA_DIR = '../data/raw'  # Adjust if your data is elsewhere
BATCH_SIZE = 32
NUM_WORKERS = 0  # Set to 0 for Windows compatibility in notebooks sometimes
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {DEVICE}")

# Create data loaders
train_loader, val_loader, class_to_idx = create_data_loaders(
    data_dir=DATA_DIR,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)

print(f"Classes: {class_to_idx}")
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")



### Visualizing Samples

Let's look at some images from our data loader to verify they are loaded correctly and see what the transforms look like.
We apply normalization for training, so we need to denormalize for visualization.


In [ ]:

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    # Denormalize
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

# Get a batch of training data
inputs, classes = next(iter(train_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs[:4])
idx_to_class = {v: k for k, v in class_to_idx.items()}
titles = [idx_to_class[x.item()] for x in classes[:4]]

plt.figure(figsize=(15, 5))
imshow(out, title=titles)



## 2. Model Architecture

We use **ResNet50** as our backbone.

**Why ResNet50?**
1.  **Transfer Learning**: It's pre-trained on ImageNet, meaning it already knows how to extract features (edges, textures, shapes). We only need to fine-tune it for masks.
2.  **Performance**: It offers a good balance between accuracy and computational cost.
3.  **Residual Connections**: Solves the vanishing gradient problem, allowing for deeper networks.

We replace the final fully connected layer to output our 3 classes: `with_mask`, `without_mask`, `mask_weared_incorrect`.


In [ ]:

model = create_model(model_name='resnet50', num_classes=len(class_to_idx), pretrained=True)
model = model.to(DEVICE)
print("Model created.")



## 3. Training

We use the `Trainer` class from `src.train`.

**Hyperparameters:**
- **Loss Function**: `CrossEntropyLoss` (standard for multi-class classification).
- **Optimizer**: `Adam` (adaptive learning rates, generally converges faster than SGD).
- **Learning Rate**: `1e-3` (good starting point for Adam).
- **Scheduler**: `ReduceLROnPlateau` (lowers LR if validation loss stops improving).


In [ ]:

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Setup training components
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# Config for trainer
config = {
    'model_name': 'resnet50',
    'epochs': 5,  # Reduced for demonstration
    'batch_size': BATCH_SIZE,
    'learning_rate': 0.001,
    'checkpoint_dir': '../models/checkpoints_notebook',
    'results_dir': '../results/notebook_metrics',
    'save_frequency': 1,
    'patience': 3
}

os.makedirs(config['checkpoint_dir'], exist_ok=True)
os.makedirs(config['results_dir'], exist_ok=True)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    class_to_idx=class_to_idx,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=DEVICE,
    config=config
)


In [ ]:

# Run training
# Note: This might take a while depending on your hardware.
# If you want to skip training, you can load the pre-trained model in the next section.
history = trainer.train()



## 4. Evaluation

Let's inspect the training history and evaluate the model on the validation set.


In [ ]:

# Plot history
plot_training_history(history)



## 5. Inference

Now we can use our trained model to make predictions on new images.


In [ ]:

# Load best model
# If training was skipped, point this to the pre-trained model
best_model_path = os.path.join(config['checkpoint_dir'], 'best_model.pth')
if not os.path.exists(best_model_path):
    print("Notebook checkpoint not found, trying main checkpoints...")
    best_model_path = '../models/checkpoints/best_model.pth'

if os.path.exists(best_model_path):
    print(f"Loading model from {best_model_path}")
    predictor = FaceMaskPredictor(model_path=best_model_path, device=DEVICE)

    # Pick a random image from validation set to test
    import random
    val_dataset = val_loader.dataset
    rand_idx = random.randint(0, len(val_dataset)-1)
    
    # We need to access the original image path. 
    # FaceMaskDataset stores samples as a list of dicts with 'image_path'
    sample = val_dataset.samples[rand_idx]
    img_path = sample['image_path']
    print(f"Testing on: {img_path}")

    predictor.visualize_prediction(img_path)
else:
    print("No model checkpoint found. Please train the model or provide a checkpoint.")



## Conclusion

We have successfully built a face mask detection system.
- **Preprocessing**: Resized to 224x224, normalized.
- **Model**: ResNet50 achieved good accuracy.
- **Next Steps**: Deploy this model using a web framework like Flask or Streamlit, or integrate it into a video stream.
